# PYNQ

In [1]:
import time
import numpy as np
from scipy import signal
from pynq import Overlay, allocate

# ol = Overlay("../overlay/system_fmradio_receiver_pynqzu.bit")
ol = Overlay("../overlay/system_2020.2-128.bit")

dma = ol.axi_dma_0
hw_fir_1 = ol.fir_complex_0
hw_fir_2 = ol.fir_real_0
hw_discrim = ol.fm_discrim_0

len_in = 1024 * 1500
len_out = int(len_in / 50)

filter_factor = 1

lpf_b1 = signal.firwin(64, 200e3/(float(2.4e6)/2))
lpf_b2 = signal.firwin(64, 12e3/(float(2.4e6)/10/2))
c1 = np.array(lpf_b1 * filter_factor, dtype=np.float32)
c2 = np.array(lpf_b2 * filter_factor, dtype=np.float32)

# 1.2) allocate buffer

coef_buffer_1 = allocate(shape=(64,), dtype=np.float32)
coef_buffer_2 = allocate(shape=(64,), dtype=np.float32)

input_buffer1 = allocate(shape=(len_in,), dtype=np.complex64)
# input_buffer2 = allocate(shape=(len_in,), dtype=np.complex64)
output_buffer1 = allocate(shape=(len_out,), dtype=np.float32)
# output_buffer2 = allocate(shape=(len_out,), dtype=np.float32)

# 1.3) init coef buffer

np.copyto(coef_buffer_1, c1)
np.copyto(coef_buffer_2, c2)

# 1.4) config phys addr

hw_fir_1.register_map.coef_1 = coef_buffer_1.physical_address
hw_fir_2.register_map.coef_1 = coef_buffer_2.physical_address
hw_fir_1.register_map.load_coef = 1
hw_fir_2.register_map.load_coef = 1

def ip_start():
    hw_fir_1.write(0x00, 0x01)
    hw_fir_2.write(0x00, 0x01)
    hw_discrim.write(0x00, 0x01)

# Async

In [2]:
from rtlsdr import RtlSdr

sdr = RtlSdr()

fc = 92_700_000    # center frequency
fs = 2_400_000    # sample rate
# 48000
# 50 times

sdr.sample_rate = fs  # Hz
sdr.center_freq = fc  # Hz
sdr.gain = 4

Found Rafael Micro R820T/2 tuner


In [3]:
# increase usb memory buffer
!echo 0 > /sys/module/usbcore/parameters/usbfs_memory_mb

In [4]:
from IPython.display import Audio
import ipywidgets as widgets
import nest_asyncio
nest_asyncio.apply()
import asyncio

## Callback Implementation

In [ ]:
from rtlsdr.helpers import limit_time
from IPython.display import Audio

flag = 0

@limit_time(10)
def read_callback(data, context):
    global flag
    
    # convert received data to array
    arr = np.ctypeslib.as_array(data)

    # convert the data type of the array to complex128
    samples = arr.astype(np.float64).view(np.complex128)
    
    # copy the samples to the input buffer
    np.copyto(input_buffer1, samples)
    
    # make sure the dma is idle
    if flag == 1:
        dma.sendchannel.wait()
        dma.recvchannel.wait()
    else:
        flag = 1
    
    # start fir and discrim ip
    ip_start()
    
    # send to DMA
    # FIR Complex
    # FM Discrim
    # FIR Real
    dma.sendchannel.transfer(input_buffer1)
    dma.recvchannel.transfer(output_buffer1)
        
    # play the audio!
    # make sure to disable automatic normalization
    # and manually normalize the array to [-1, 1]
    display(Audio(output_buffer1 / 50, autoplay=True, rate=48000, normalize=False))

# the data type of read_bytes is [I_uint8[0], Q_uint8[0], I_uint8[1], ...]
sdr.read_bytes_async(read_callback, num_bytes=int(len_in*2))

## Asyncio Implementation

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
from IPython.display import Audio

flag = 0

async def streaming():
    async for data in sdr.stream(num_samples_or_bytes=1024 * 3000, format='bytes', loop=None):
        # do something with samples
        global flag, freq

        # convert received data to array
        arr = np.ctypeslib.as_array(data)

        # convert the data type of the array to complex128
        samples = arr.astype(np.float64).view(np.complex128)

        # copy the samples to the input buffer
        np.copyto(input_buffer1, samples)

        # make sure the dma is idle
        if flag == 1:
            dma.sendchannel.wait()
            dma.recvchannel.wait()
        else:
            flag = 1

        # start fir and discrim ip
        ip_start()

        # send to DMA
        # FIR Complex
        # FM Discrim
        # FIR Real
        dma.sendchannel.transfer(input_buffer1)
        dma.recvchannel.transfer(output_buffer1)

        # play the audio!
        # make sure to disable automatic normalization
        # and manually normalize the array to [-1, 1]
        display(Audio(output_buffer1 / 50, autoplay=True, rate=48000, normalize=False))
        
    # to stop streaming:
    await sdr.stop()
    
loop = asyncio.get_event_loop()
loop.run_until_complete(streaming())

In [ ]:
sdr.stop()

# Hide the play bar

In [ ]:
# un-hide the play bar from IPython.display.Audio
display_audio_css = """
<style>
div.lm-Widget.p-Widget.lm-Panel.p-Panel.jp-OutputArea-child {
  display: block;
}
audio { display: none }
</style>
"""
display(HTML(display_audio_css))

# Asyncio to change fc with a preset

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
from IPython.display import Audio

flag = 0

freq = 91_000_000

async def streaming():
    async for data in sdr.stream(num_samples_or_bytes=1024 * 3000, format='bytes', loop=None):
        # do something with samples
        global flag, freq

        # convert received data to array
        arr = np.ctypeslib.as_array(data)

        # convert the data type of the array to complex128
        samples = arr.astype(np.float64).view(np.complex128)

        # copy the samples to the input buffer
        np.copyto(input_buffer1, samples)

        # debug
        sdr.center_freq = freq
        print(sdr.center_freq)
        freq = freq + 100_000

        # make sure the dma is idle
        if flag == 1:
            dma.sendchannel.wait()
            dma.recvchannel.wait()
        else:
            flag = 1

        # start fir and discrim ip
        ip_start()

        # send to DMA
        # FIR Complex
        # FM Discrim
        # FIR Real
        dma.sendchannel.transfer(input_buffer1)
        dma.recvchannel.transfer(output_buffer1)

        # play the audio!
        # make sure to disable automatic normalization
        # and manually normalize the array to [-1, 1]
        display(Audio(output_buffer1 / 50, autoplay=True, rate=48000, normalize=False))
        
    # to stop streaming:
    await sdr.stop()
    
loop = asyncio.get_event_loop()
loop.run_until_complete(streaming())

In [ ]:
sdr.stop()

# Asyncio to read slider in real time

## TBD

In [ ]:
sdr.stop()

# Asyncio to change fc by a slider

In [ ]:
%gui asyncio

## Ex1: async def wait_for_change)

```
Input In [12], in change_val()
     87 val = await wait_for_change(slider, 'value')
     88 # sdr.center_freq = val
---> 89 print(val.result())

File /usr/lib/python3.10/asyncio/futures.py:198, in Future.result(self)
    196     raise exc
    197 if self._state != _FINISHED:
--> 198     raise exceptions.InvalidStateError('Result is not ready.')
    199 self.__log_traceback = False
    200 if self._exception is not None:

InvalidStateError: Result is not ready.
```

In [ ]:
import ipywidgets as widgets
import nest_asyncio
nest_asyncio.apply()
import asyncio

flag = 0

# Create an IntSlider widget
slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=80_000_000,    # Minimum value
    max=110_000_000,  # Maximum value
    step=100_000,   # Step size
    description='Center Freq:',  # Description displayed next to the slider
    orientation='horizontal', # Orientation of the slider
    # disabled=False,
    # continuous_update=False,
    # readout=True,
    # readout_format='d'
)

out = widgets.Output()

it_num = 10

async def streaming():
    it = 0
    async for data in sdr.stream(num_samples_or_bytes=1024 * 3000, format='bytes', loop=None):        
        # do something with samples
        global flag

        # convert received data to array
        arr = np.ctypeslib.as_array(data)

        # convert the data type of the array to complex128
        samples = arr.astype(np.float64).view(np.complex128)

        # copy the samples to the input buffer
        np.copyto(input_buffer1, samples)

        # debug
        # print(slider.value)

        # make sure the dma is idle
        if flag == 1:
            dma.sendchannel.wait()
            dma.recvchannel.wait()
        else:
            flag = 1

        # start fir and discrim ip
        ip_start()
        
        # send to DMA
        dma.sendchannel.transfer(input_buffer1)
        dma.recvchannel.transfer(output_buffer1)

        # play the audio!
        # make sure to disable automatic normalization
        # and manually normalize the array to [-1, 1]
        display(Audio(output_buffer1 / 50, autoplay=True, rate=48000, normalize=False))
        
        it = it + 1
        if it == it_num:
            break
            
async def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future
            
async def show_val():
    for i in range(it_num):
        print('awaiting slider change...')
        val = await wait_for_change(slider, 'value')
        # sdr.center_freq = val
        print('async function continued with value ' + str(val) + '\n')

async def change_val():
    # val = asyncio.Future()
    for i in range(it_num):
        print('awaiting slider change...')
        val = await wait_for_change(slider, 'value')
        # sdr.center_freq = val
        print(val.result())
        # print(dir(val))
        # print(type(val))
        # print("changed sdr.center_freq to %d" % val)
    
async def main():
    print("main task started")
    
    print("task 1 created")
    task1 = asyncio.create_task(streaming())
    
    print("task 2 created")
    # task2 = asyncio.create_task(show_val())
    # task2 = asyncio.ensure_future(show_val())
    task2 = asyncio.create_task(change_val())
    
    print("awaiting task 1")
    await task1
    print("task 1 finished")
    
    print("awaiting task 2")
    await task2
    print("task 2 finished")
    
    print("main task finished")
    
# Display the slider widget
display(slider)
display(out)

asyncio.run(main())

# asyncio.create_task(streaming())

## Ex2 use def wait_for_change suggested by official doc

Only after finishing playing the audio will the change slider be executed

In [ ]:
import ipywidgets as widgets
import nest_asyncio
nest_asyncio.apply()
import asyncio

flag = 0

# Create an IntSlider widget
slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=80_000_000,    # Minimum value
    max=110_000_000,  # Maximum value
    step=100_000,   # Step size
    description='Center Freq:',  # Description displayed next to the slider
    orientation='horizontal', # Orientation of the slider
    # disabled=False,
    # continuous_update=False,
    # readout=True,
    # readout_format='d'
)

out = widgets.Output()

it_num = 10

async def streaming():
    it = 0
    async for data in sdr.stream(num_samples_or_bytes=1024 * 3000, format='bytes', loop=None):        
        # do something with samples
        global flag

        # convert received data to array
        arr = np.ctypeslib.as_array(data)

        # convert the data type of the array to complex128
        samples = arr.astype(np.float64).view(np.complex128)

        # copy the samples to the input buffer
        np.copyto(input_buffer1, samples)

        # debug
        # print(slider.value)

        # make sure the dma is idle
        if flag == 1:
            dma.sendchannel.wait()
            dma.recvchannel.wait()
        else:
            flag = 1

        # start fir and discrim ip
        ip_start()
        
        # send to DMA
        dma.sendchannel.transfer(input_buffer1)
        dma.recvchannel.transfer(output_buffer1)

        # play the audio!
        # make sure to disable automatic normalization
        # and manually normalize the array to [-1, 1]
        display(Audio(output_buffer1 / 50, autoplay=True, rate=48000, normalize=False))
        
        it = it + 1
        if it == it_num:
            break
            
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

async def change_val():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(slider, 'value')
        out.append_stdout('async function continued with value ' + str(x) + '\n')
    
async def main():
    print("main task started")
    
    print("task 1 created")
    task1 = asyncio.create_task(streaming())
    
    print("task 2 created")
    asyncio.ensure_future(change_val())
    print("awaiting task 1")
    await task1
    print("task 1 finished")
    
    print("main task finished")
    
# Display the slider widget
display(slider)
display(out)

asyncio.run(main())

# Test slider

## Ex3 directly print in async for loop
no use

In [ ]:
import ipywidgets as widgets
import nest_asyncio
nest_asyncio.apply()
import asyncio
from IPython.display import Audio

flag = 0

# Create an IntSlider widget
slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=80_000_000,    # Minimum value
    max=110_000_000,  # Maximum value
    step=100_000,   # Step size
    description='Center Freq:',  # Description displayed next to the slider
    orientation='horizontal', # Orientation of the slider
    disabled=False,
    continuous_update=False,
    readout=True,
    readout_format='d'
)

async def streaming():
    async for data in sdr.stream(num_samples_or_bytes=1024 * 3000, format='bytes', loop=None):
        # debug
        print(slider.value)
        
    # to stop streaming:
    await sdr.stop()
    
display(slider)
    
loop = asyncio.get_event_loop()
loop.run_until_complete(streaming())

# Test future

## Ex4 suggested by official doc

In [ ]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

from ipywidgets import IntSlider, Output
# Create an IntSlider widget
slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=80_000_000,    # Minimum value
    max=110_000_000,  # Maximum value
    step=100_000,   # Step size
    description='Center Freq:',  # Description displayed next to the slider
    orientation='horizontal', # Orientation of the slider
    # disabled=False,
    # continuous_update=False,
    # readout=True,
    # readout_format='d'
)
out = Output()

async def f():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(slider, 'value')
        out.append_stdout('async function continued with value ' + str(x) + '\n')
asyncio.ensure_future(f())

display(slider)
display(out)

## Ex5 odd behavior when ensure_future is nested in main
The outputs will be shown twice

In [ ]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

from ipywidgets import IntSlider, Output
# Create an IntSlider widget
slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=80_000_000,    # Minimum value
    max=110_000_000,  # Maximum value
    step=100_000,   # Step size
    description='Center Freq:',  # Description displayed next to the slider
    orientation='horizontal', # Orientation of the slider
    # disabled=False,
    # continuous_update=False,
    # readout=True,
    # readout_format='d'
)
out = Output()

async def f():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(slider, 'value')
        out.append_stdout('async function continued with value ' + str(x) + '\n')


async def main():
    print("main task started")
    
    task = asyncio.ensure_future(f())
    # await task
    
    print("main task finished")
    
# Display the slider widget
display(slider)
display(out)

# asyncio.run(main())

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

display(slider)
display(out)

## Ex6 display(, display_id=True).update

It works! But only in sync for loop inside main

In [ ]:
import ipywidgets as widgets
import nest_asyncio
nest_asyncio.apply()
import asyncio
from IPython.display import Audio

flag = 0

# Create an IntSlider widget
slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=80_000_000,    # Minimum value
    max=110_000_000,  # Maximum value
    step=100_000,   # Step size
    description='Center Freq:',  # Description displayed next to the slider
    orientation='horizontal', # Orientation of the slider
    disabled=False,
    continuous_update=False,
    readout=True,
    readout_format='d'
)

async def streaming(slider, output):
    for _ in range(100):
        # debug
        # print(slider.value)
        await asyncio.sleep(1)
        output.update('test ' + str(slider.value))
    
output = display("tbd", display_id=True)
display(slider)
    
loop = asyncio.get_event_loop()
loop.run_until_complete(streaming(slider, output))


In [ ]:
import ipywidgets as widgets
import nest_asyncio
nest_asyncio.apply()
import asyncio
from IPython.display import Audio

flag = 0

# Create an IntSlider widget
slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=80_000_000,    # Minimum value
    max=110_000_000,  # Maximum value
    step=100_000,   # Step size
    description='Center Freq:',  # Description displayed next to the slider
    orientation='horizontal', # Orientation of the slider
    disabled=False,
    continuous_update=False,
    readout=True,
    readout_format='d'
)

async def streaming(slider, output):
    for _ in range(10):
        # debug
        print(slider.value)
        await asyncio.sleep(1)
        output.update('test ' + str(slider.value))
    
output = display("tbd", display_id=True)
display(slider)
    
asyncio.create_task(streaming(slider, output))
# asyncio.run(streaming(slider, output))

# Call back change fc will throw USB error

In [ ]:
import ipywidgets as widgets
from rtlsdr.helpers import limit_time
from IPython.display import Audio

slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=90_000_000,    # Minimum value
    max=100_000_000,  # Maximum value
    step=100_000,   # Step size
    description='Center Freq:',  # Description displayed next to the slider
    orientation='horizontal',  # Orientation of the slider
    disabled=False,
    continuous_update=False,
    readout=True,
    readout_format='d'
)

@limit_time(10)
def read_callback(data, context):
    global flag
    
    print(slider.value)
    # sdr.center_freq = slider.value
        
    # convert received data to array
    arr = np.ctypeslib.as_array(data)

    # convert the data type of the array to complex128
    samples = arr.astype(np.float64).view(np.complex128)
    
    # copy the samples to the input buffer
    np.copyto(input_buffer1, samples)
    
    # make sure the dma is idle
    if flag == 1:
        dma.sendchannel.wait()
        dma.recvchannel.wait()
    else:
        flag = 1
    
    # start fir and discrim ip
    ip_start()
    
    # send to DMA
    dma.sendchannel.transfer(input_buffer1)
    dma.recvchannel.transfer(output_buffer1)
        
    # play the audio!
    # make sure to disable automatic normalization
    # and manually normalize the array to [-1, 1]
    display(Audio(output_buffer1 / 50, autoplay=True, rate=48000, normalize=False))

display(slider)
    
# the data type of read_bytes is [I_uint8[0], Q_uint8[0], I_uint8[1], ...]
sdr.read_bytes_async(read_callback, num_bytes=int(len_in*2))

# Final Try

In [5]:
import ipywidgets as widgets
import nest_asyncio
nest_asyncio.apply()
import asyncio
from jupyter_ui_poll import ui_events

flag = 0

it_num = 10

slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=90_000_000,    # Minimum value
    max=100_000_000,  # Maximum value
    step=100_000,   # Step size
    description='Center Freq:',  # Description displayed next to the slider
    orientation='horizontal',  # Orientation of the slider
)

async def streaming():
    it = 0
    async for data in sdr.stream(num_samples_or_bytes=1024 * 3000, format='bytes', loop=None):        
        # do something with samples
        global flag

        # convert received data to array
        arr = np.ctypeslib.as_array(data)

        # convert the data type of the array to complex128
        samples = arr.astype(np.float64).view(np.complex128)

        # copy the samples to the input buffer
        np.copyto(input_buffer1, samples)

        # make sure the dma is idle
        if flag == 1:
            dma.sendchannel.wait()
            dma.recvchannel.wait()
        else:
            flag = 1

        # start fir and discrim ip
        ip_start()
        
        # send to DMA
        dma.sendchannel.transfer(input_buffer1)
        dma.recvchannel.transfer(output_buffer1)
        
        st = time.time()
        # poll ui events
        with ui_events() as poll:
            poll(1)
            sdr.center_freq = slider.value
        et = time.time()
        print("%d s", (et - st))

        # play the audio!
        # make sure to disable automatic normalization
        # and manually normalize the array to [-1, 1]
        display(Audio(output_buffer1 / 50, autoplay=True, rate=48000, normalize=False))
        
        it = it + 1
        if it == it_num:
            break
    
display(slider)

asyncio.run(streaming())

IntSlider(value=92700000, description='Center Freq:', max=100000000, min=90000000, step=100000)

Allocating 15 zero-copy buffers


%d s 0.050611019134521484


%d s 0.052414655685424805


%d s 0.04909801483154297


...

%d s 0.04861187934875488


%d s 0.04881906509399414


%d s 0.04857230186462402


%d s 0.048581838607788086


%d s 0.04871511459350586


# Observe

In [14]:
from ipywidgets import IntSlider
from IPython.display import display

slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=80_000_000, 
    max=110_000_000, 
    step=100_000, 
    description='Center Freq:',
    orientation='horizontal',
)

out = widgets.Output()

def on_slider_change(change):
    new_center_freq = change['new']
    # Update the sdr.center_freq parameter with the new value
    sdr.center_freq = new_center_freq
    
    with out:
        print("sdr.center_freq is changed to %d" % sdr.center_freq)

slider.observe(on_slider_change, names='value')
display(slider, out)

IntSlider(value=92700000, description='Center Freq:', max=110000000, min=80000000, step=100000)

Output()

In [5]:
import ipywidgets as widgets
import nest_asyncio
nest_asyncio.apply()
import asyncio

flag = 0
it_num = 10

async def streaming():
    it = 0
    async for data in sdr.stream(num_samples_or_bytes=1024 * 3000, format='bytes', loop=None):        
        # do something with samples
        global flag

        # convert received data to array
        arr = np.ctypeslib.as_array(data)

        # convert the data type of the array to complex128
        samples = arr.astype(np.float64).view(np.complex128)

        # copy the samples to the input buffer
        np.copyto(input_buffer1, samples)

        # debug
        # val = await wait_for_change(slider, 'value')
        # print(val)

        # make sure the dma is idle
        if flag == 1:
            dma.sendchannel.wait()
            dma.recvchannel.wait()
        else:
            flag = 1

        # start fir and discrim ip
        ip_start()
        
        # send to DMA
        dma.sendchannel.transfer(input_buffer1)
        dma.recvchannel.transfer(output_buffer1)

        # play the audio!
        # make sure to disable automatic normalization
        # and manually normalize the array to [-1, 1]
        display(Audio(output_buffer1 / 50, autoplay=True, rate=48000, normalize=False))
        
        it = it + 1
        if it == it_num:
            break

async def main():
    print("task started")
    task = asyncio.create_task(streaming())
    await task
    print("task finished")
            
def on_slider_change(change):
    new_center_freq = change['new']
    # Update the sdr.center_freq parameter with the new value
    sdr.center_freq = new_center_freq
    with out:
        print("sdr.center_freq is changed to %d" % sdr.center_freq)


slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=80_000_000, 
    max=110_000_000, 
    step=100_000, 
    description='Center Freq:',
    orientation='horizontal',
)

out = widgets.Output()

slider.observe(on_slider_change, names='value')

display(slider, out)
    
asyncio.run(main())

IntSlider(value=92700000, description='Center Freq:', max=110000000, min=80000000, step=100000)

Output()

task started


Allocating 15 zero-copy buffers


task finished


In [14]:
import ipywidgets as widgets
import nest_asyncio
nest_asyncio.apply()
import asyncio
import concurrent.futures

flag = 0
it_num = 10

async def streaming():
    it = 0
    async for data in sdr.stream(num_samples_or_bytes=1024 * 3000, format='bytes', loop=None):        
        # do something with samples
        global flag

        # convert received data to array
        arr = np.ctypeslib.as_array(data)

        # convert the data type of the array to complex128
        samples = arr.astype(np.float64).view(np.complex128)

        # copy the samples to the input buffer
        np.copyto(input_buffer1, samples)

        # debug
        # val = await wait_for_change(slider, 'value')
        # print(val)

        # make sure the dma is idle
        if flag == 1:
            dma.sendchannel.wait()
            dma.recvchannel.wait()
        else:
            flag = 1

        # start fir and discrim ip
        ip_start()
        
        # send to DMA
        dma.sendchannel.transfer(input_buffer1)
        dma.recvchannel.transfer(output_buffer1)

        # play the audio!
        # make sure to disable automatic normalization
        # and manually normalize the array to [-1, 1]
        display(Audio(output_buffer1 / 50, autoplay=True, rate=48000, normalize=False))
        
        it = it + 1
        if it == it_num:
            break
            
def on_slider_change(change):
    new_center_freq = change['new']
    # Update the sdr.center_freq parameter with the new value
    sdr.center_freq = new_center_freq
    with out:
        print("sdr.center_freq is changed to %d" % sdr.center_freq)

slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=80_000_000, 
    max=110_000_000, 
    step=100_000, 
    description='Center Freq:',
    orientation='horizontal',
)
        
out = widgets.Output()
    
async def main():
    # loop = asyncio.get_running_loop()
    print("main task started")
    
    task = asyncio.create_task(streaming())
    
    coro1 = asyncio.to_thread(slider.observe, on_slider_change, 'value')
    
    coro2 = asyncio.to_thread(display, slider, out)
    
    await coro1
    
    await coro2
    
    await task

    print("main task finished")
    
asyncio.run(main())

main task started


/usr/lib/python3.10/ast.py:268: RuntimeWarning: coroutine 'to_thread' was never awaited
  for item in field:

KeyboardInterrupt

